In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -U autogluon > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


In [2]:
import pandas as pd
from autogluon.tabular import TabularPredictor
import os


In [11]:
data_path = '/kaggle/input/feedback-prize-effectiveness'
train_csv_path = os.path.join(data_path, 'train.csv')
train_df = pd.read_csv(train_csv_path)
print("Dữ liệu huấn luyện có", len(train_df), "hàng.")


Dữ liệu huấn luyện có 36765 hàng.


In [4]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train_df, test_size = 0.2, random_state = 3, stratify = train_df['discourse_effectiveness'] )

In [5]:
label = 'discourse_effectiveness'

In [6]:
predictor = TabularPredictor(
    label=label,
    eval_metric='log_loss'
).fit(
    train_data = train_data,
    time_limit = 600
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250630_145108"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.81 GB / 31.35 GB (95.1%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions

[1000]	valid_set's multi_logloss: 0.652658


	-0.6438	 = Validation score   (-log_loss)
	70.81s	 = Training   runtime
	0.98s	 = Validation runtime
Fitting model: RandomForestGini ... Training model for up to 329.32s of the 329.32s of remaining time.
	-0.8224	 = Validation score   (-log_loss)
	168.01s	 = Training   runtime
	0.39s	 = Validation runtime
Fitting model: RandomForestEntr ... Training model for up to 160.16s of the 160.16s of remaining time.
	-0.8151	 = Validation score   (-log_loss)
	136.99s	 = Training   runtime
	0.35s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 21.98s of the 21.97s of remaining time.
	Many features detected (9055), dynamically setting 'colsample_bylevel' to 0.11043622308117063 to speed up training (Default = 1).
	To disable this functionality, explicitly specify 'colsample_bylevel' in the model hyperparameters.
	Ran out of time, early stopping on iteration 1.
	-1.0825	 = Validation score   (-log_loss)
	15.68s	 = Training   runtime
	0.51s	 = Validation runtime
Fitting mo

In [12]:
y_true = test_data[label]
y_pred = predictor.predict(test_data)
y_proba = predictor.predict_proba(test_data)


print("AutoGluon Evaluation:")
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'log_loss', 'f1_macro', 'f1_weighted', 'roc_auc_ovr_macro', 'roc_auc_ovr_weighted'])



AutoGluon Evaluation:


,model,score_test,accuracy,log_loss,f1_macro,f1_weighted,roc_auc_ovr,roc_auc_ovr_weighted,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.607523,0.733850,-0.607523,0.639545,0.710914,0.850435,0.835072,-0.611240,log_loss,5.570127,1.374568,241.117007,0.004860,0.001756,0.292353,2,True,9
1,LightGBM,-0.630263,0.734938,-0.630263,0.650482,0.716398,0.845190,0.829035,-0.643773,log_loss,2.972107,0.975409,70.811756,2.972107,0.975409,70.811756,1,True,5
2,LightGBMXT,-0.726364,0.685299,-0.726364,0.582364,0.659501,0.796201,0.773060,-0.742729,log_loss,2.447841,0.568084,65.796991,2.447841,0.568084,65.796991,1,True,4
3,NeuralNetFastAI,-0.731691,0.694546,-0.731691,0.624737,0.684699,0.808582,0.794917,-0.725978,log_loss,0.287795,0.049090,33.025633,0.287795,0.049090,33.025633,1,True,3
4,RandomForestEntr,-0.801031,0.645451,-0.801031,0.456616,0.578240,0.758484,0.733711,-0.815078,log_loss,2.305365,0.348313,136.987265,2.305365,0.348313,136.987265,1,True,7
5,RandomForestGini,-0.805086,0.645723,-0.805086,0.462493,0.581448,0.753169,0.729884,-0.822409,log_loss,2.757589,0.393684,168.012482,2.757589,0.393684,168.012482,1,True,6
6,CatBoost,-1.082272,0.604651,-1.082272,0.357289,0.507299,0.550240,0.558968,-1.082466,log_loss,1.465542,0.505817,15.675976,1.465542,0.505817,15.675976,1,True,8
7,KNeighborsUnif,-2.341401,0.595131,-2.341401,0.445950,0.552527,0.629957,0.624845,-2.335141,log_loss,25.440251,7.240264,10.780032,25.440251,7.240264,10.780032,1,True,1
8,KNeighborsDist,-2.348156,0.577995,-2.348156,0.456147,0.551428,0.635092,0.627424,-2.352427,log_loss,25.011472,7.205302,10.486506,25.011472,7.205302,10.486506,1,True,2


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    log_loss,
    f1_score,
    roc_auc_score,
)
from sklearn.preprocessing import LabelEncoder

In [ ]:
TRAIN_CSV_PATH = '/kaggle/input/feedback-prize-effectiveness/train.csv'
TRAIN_ESSAYS_PATH = '/kaggle/input/feedback-prize-effectiveness/train/'

In [ ]:
def create_dummy_data():
    """
    Creates dummy data files for local testing if they don't exist.
    This simulates the Kaggle environment.
    """
    print("Creating dummy data for local execution...")

    # Create dummy train.csv
    if not os.path.exists(TRAIN_CSV_PATH):
        train_data = {
            'discourse_id': [f'd{i}' for i in range(100)],
            'essay_id': [f'essay_{i % 10}' for i in range(100)],
            'discourse_text': [
                'This is a sample lead.', 'The author makes a claim.', 'Here is some evidence.',
                'This is a concluding statement.', 'Another position is taken here.'
            ] * 20,
            'discourse_type': ['Lead', 'Claim', 'Evidence', 'Concluding Statement', 'Position'] * 20,
            'discourse_effectiveness': ['Adequate', 'Effective', 'Ineffective', 'Adequate', 'Effective'] * 20
        }
        train_df = pd.DataFrame(train_data)
        train_df.to_csv(TRAIN_CSV_PATH, index=False)
        print(f"'{TRAIN_CSV_PATH}' created.")

    # Create dummy essay text files
    if not os.path.exists(TRAIN_ESSAYS_PATH):
        os.makedirs(TRAIN_ESSAYS_PATH)
        for i in range(10):
            essay_id = f'essay_{i}'
            essay_text = (
                f"This is the full text for essay {essay_id}. "
                "It contains various discourse elements that are analyzed. "
                "The student argues a point, provides evidence, and concludes the argument. "
                "The quality of these elements varies."
            )
            with open(os.path.join(TRAIN_ESSAYS_PATH, f'{essay_id}.txt'), 'w') as f:
                f.write(essay_text)
        print(f"'{TRAIN_ESSAYS_PATH}' directory and dummy essays created.")

In [ ]:
def load_data(train_csv_path, essays_path):
    """
    Loads the training data from the CSV and merges it with the full essay texts.

    Args:
        train_csv_path (str): Path to the train.csv file.
        essays_path (str): Path to the directory containing essay .txt files.

    Returns:
        pandas.DataFrame: A DataFrame with the combined data.
    """
    print("Loading data...")
    # Load the main training data
    try:
        df = pd.read_csv(train_csv_path)
    except FileNotFoundError:
        print(f"Error: Training CSV not found at '{train_csv_path}'.")
        return None

    # Load essay texts into a dictionary
    essay_texts = {}
    for filename in os.listdir(essays_path):
        if filename.endswith('.txt'):
            essay_id = filename.split('.')[0]
            with open(os.path.join(essays_path, filename), 'r') as f:
                essay_texts[essay_id] = f.read()

    # Map the essay texts to the DataFrame
    df['essay_full_text'] = df['essay_id'].map(essay_texts)
    print("Data loading complete.")
    return df

In [ ]:
def feature_engineering(df):
    """
    Creates a combined text feature for the model.

    Args:
        df (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The DataFrame with the new 'full_context_text' feature.
    """
    print("Performing feature engineering...")
    # Combine the discourse text, type, and the full essay for context
    # Using a separator to give the model a hint about the different parts
    df['full_context_text'] = (
        df['discourse_type'] + ' [SEP] ' +
        df['discourse_text'] + ' [SEP] ' +
        df['essay_full_text']
    )
    print("Feature engineering complete.")
    return df

In [ ]:
if __name__ == "__main__":
    # Create dummy files if they don't exist (for local runs)
    if not (os.path.exists(TRAIN_CSV_PATH) and os.path.exists(TRAIN_ESSAYS_PATH)):
        create_dummy_data()

    # 1. Load and Prepare Data
    train_df = load_data(TRAIN_CSV_PATH, TRAIN_ESSAYS_PATH)

    if train_df is not None:
        train_df = feature_engineering(train_df)

        # Define features (X) and target (y)
        X = train_df['full_context_text']
        y_raw = train_df['discourse_effectiveness']

        # 2. Encode Target Labels
        # The labels need to be converted from strings to integers (0, 1, 2)
        print("Encoding target labels...")
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y_raw)
        # The mapping will be: Adequate -> 0, Effective -> 1, Ineffective -> 2
        # We can see the classes with `label_encoder.classes_`
        class_names = label_encoder.classes_
        print(f"Labels encoded. Class mapping: {dict(zip(class_names, range(len(class_names))))}")


        # 3. Split Data into Training and Validation Sets
        print("Splitting data into training and validation sets (80/20 split)...")
        X_train, X_val, y_train, y_val = train_test_split(
            X, y,
            test_size=0.2,
            random_state=42,
            stratify=y  # Stratify to maintain class distribution in train/test splits
        )
        print(f"Training set size: {len(X_train)}")
        print(f"Validation set size: {len(X_val)}")

        # 4. Define the Model Pipeline
        # A pipeline makes it easy to chain preprocessing and modeling steps.
        # Step 1: TfidfVectorizer - Converts text to a matrix of TF-IDF features.
        # Step 2: LogisticRegression - A simple, effective baseline model for text classification.
        print("Defining the model pipeline...")
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(
                ngram_range=(1, 3), # Use unigrams, bigrams, and trigrams
                max_features=10000, # Limit the number of features to the top 10k
                stop_words='english'
            )),
            ('clf', LogisticRegression(
                solver='liblinear', # Good for smaller datasets
                random_state=42,
                C=1.0 # Regularization strength
            ))
        ])

        # 5. Train the Model
        print("Training the model...")
        pipeline.fit(X_train, y_train)
        print("Model training complete.")

        # 6. Make Predictions on the Validation Set
        print("Making predictions on the validation set...")
        y_pred = pipeline.predict(X_val)
        y_pred_proba = pipeline.predict_proba(X_val)
        print("Predictions complete.")

        # 7. Evaluate the Model
        print("\n--- Model Evaluation Results ---")

        # Calculate metrics
        accuracy = accuracy_score(y_val, y_pred)
        loss = log_loss(y_val, y_pred_proba)
        f1_macro = f1_score(y_val, y_pred, average='macro')
        f1_weighted = f1_score(y_val, y_pred, average='weighted')
        # For multiclass, roc_auc_score needs probabilities and multi_class='ovr'
        roc_auc_ovr = roc_auc_score(y_val, y_pred_proba, multi_class='ovr', average='macro')
        roc_auc_ovr_weighted = roc_auc_score(y_val, y_pred_proba, multi_class='ovr', average='weighted')

        # Create a DataFrame for a clean display of results
        results_df = pd.DataFrame({
            'Metric': [
                'Accuracy',
                'Log Loss',
                'F1 Score (Macro)',
                'F1 Score (Weighted)',
                'ROC AUC (OVR Macro)',
                'ROC AUC (OVR Weighted)'
            ],
            'Score': [
                accuracy,
                loss,
                f1_macro,
                f1_weighted,
                roc_auc_ovr,
                roc_auc_ovr_weighted
            ]
        })

        print(results_df.to_string(index=False))
        print("--------------------------------\n")